## Importación de librerías

### Pulsos Binaurales

In [1]:
from AccelBrainBeat.brainbeat.binaural_beat import BinauralBeat
from pydub import AudioSegment
AudioSegment.converter =r"C:\Users\DiegoAlegria\Documents\Universidad\Último Semestre\Tesis\Codes\ffmpeg-4.4-full_build\bin\ffmpeg.exe"
AudioSegment.ffmpeg = r"C:\Users\DiegoAlegria\Documents\Universidad\Último Semestre\Tesis\Codes\ffmpeg-4.4-full_build\bin\ffmpeg.exe"
AudioSegment.ffprobe =r"C:\Users\DiegoAlegria\Documents\Universidad\Último Semestre\Tesis\Codes\ffmpeg-4.4-full_build\bin\ffprob.exe"
from pydub.playback import play
import time
import threading
import numpy as np
brain_beat = BinauralBeat()
# ruta=r'C:\Users\DiegoAlegria\Documents\Universidad\Último Semestre\Tesis\Codes\ffmpeg-4.4-full_build\bin'
def binaural_player(frequencies,i):
    brain_beat.save_beat(output_file_name="beat.wav",
                         frequencys=(frequencies[i][0], frequencies[i][1]),play_time=30,volume=0.5)
    beat = AudioSegment.from_file("beat.wav")
    mixed = beat.overlay(pista).fade_in(1000).fade_out(2000)-40
#     return mixed
    print('Frecuencias reproducidas: '+str(frequencies[i][0])+'-'+ str(frequencies[i][1]))
    play(mixed)
    
pista = AudioSegment.from_file("pista.wav")
frequencies={1:[250,280],
             2:[250,256],
             3:[250,259],
             4:[250,252],
             5:[250,262]}

C:\Users\DiegoAlegria\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


### Brainflow y OpenBCI

In [2]:
import argparse
import time
import numpy as np
import pandas as pd
import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations

### Clasificador

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import xgboost
from sklearn.svm import SVC
## Includes
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer


## Train Test Split
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

## Métricas
from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve
from hurst import compute_Hc
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

## Over sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN

features_final=[
 'DFA',
 'HFF',
 'Fisher Info',
 'SVD Entropy',
 'PFD',
 'Power Delta Normalized',
 'Power Theta Normalized',
 'Power Alpha Normalized',
 'Power Beta Normalized',
 'Hjorth Mobility',
 'Hjorth Complexity'
 ]
band_list = [0.5,4,7,12,30]
freq=100

### Importación de modelo

In [ ]:
import joblib
clf = joblib.load('modelo_final.pkl')

In [9]:
try:
    while True:
        print('Alive')
        time.sleep(1)

except KeyboardInterrupt:
    print("Interrupción manual activada")
    pass

Alive
Alive
Alive
Interrupción manual activada


## Playground

In [ ]:
def main():
    BoardShim.enable_dev_board_logger()
    params = BrainFlowInputParams()
    params.serial_port = 'COM3'
    board = BoardShim(0 ,params)
    board.prepare_session()
    board.start_stream()
    BoardShim.log_message(LogLevels.LEVEL_INFO.value, 'start sleeping in the main thread')
    time.sleep(10)
    data = board.get_board_data()
    board.stop_stream()
    board.release_session()

    # demo how to convert it to pandas DF and plot data
    eeg_channels = BoardShim.get_eeg_channels(0)
    df = pd.DataFrame(np.transpose(data))
    print('Data From the Board')
    display(df.head(10))

    # demo for data serialization using brainflow API, we recommend to use it instead pandas.to_csv()
    DataFilter.write_file(data, 'test.csv', 'w')  # use 'a' for append mode
    restored_data = DataFilter.read_file('test.csv')
    restored_df = pd.DataFrame(np.transpose(restored_data))
    print('Data From the File')
    display(restored_df.head(10))


if __name__ == "__main__":
    main()

## Integración final

In [4]:
df_prueba=[]
epochs=[0]
temps=[]
annots=[]

def main():
    BoardShim.enable_dev_board_logger()
    params = BrainFlowInputParams()
    params.serial_port = 'COM3'
    board = BoardShim(0 ,params)
    board.prepare_session()
    board.start_stream()
    BoardShim.log_message(LogLevels.LEVEL_INFO.value, 'start sleeping in the main thread')
    time.sleep(30)
    data = board.get_board_data()
    board.stop_stream()
    board.release_session()

    # demo how to convert it to pandas DF and plot data
    eeg_channels = BoardShim.get_eeg_channels(0)
    df = pd.DataFrame(np.transpose(data))
    df=df[[1,2]].copy()
    df.drop([0],axis=0,inplace=True)
    df.reset_index(inplace=True,drop=True)
    df2=pd.DataFrame(columns=['FpZ'])
    df2['FpZ']=(signal.resample(df2[1],3000)+signal.resample(df2[2],3000))/2
    df2['Epoch']=max(epochs)+1
    epochs.append(max(epochs)+1)
    
    temp=df2.groupby(['Epoch']).agg([pyeeg.dfa,
#                                                                               lambda x: pyeeg.hurst(x.tolist()),
                                                                          lambda x: pyeeg.hfd(x.tolist(),5),
                                                                          # lambda x: pyeeg.spectral_entropy(x.tolist(),band_list,freq,Power_Ratio=None),
                                                                          lambda x: pyeeg.fisher_info(x.tolist(),4,10),
                                                                          # lambda x: pyeeg.svd_entropy(x),
                                                                          # lambda x: pyeeg.ap_entropy(x,10,0.3*(np.std(x))),
                                                                          lambda x: pyeeg.svd_entropy(x,4,10),
                                                                          lambda x: pyeeg.bin_power(x, band_list, freq),
                                                                          lambda x: pyeeg.pfd(x, np.diff(x).tolist()),
                                                                          lambda x: pyeeg.hjorth(x.tolist(), np.diff(x).tolist())])



    temp.reset_index(inplace=True,drop=False)
    temp.columns=['epoch','DFA','HFF','Fisher Info','SVD Entropy','PSD','PFD','Hjorth']
    temp[['Power Delta'
      ,'Power Theta'
      ,'Power Alpha'
      ,'Power Beta',
      'Power Delta Normalized',
      'Power Theta Normalized',
      'Power Alpha Normalized',
      'Power Beta Normalized']]      =    temp.PSD[0][0][0],temp.PSD[0][0][1],temp.PSD[0][0][2],temp.PSD[0][0][3],temp.PSD[0][1][0],temp.PSD[0][1][1],temp.PSD[0][1][2],temp.PSD[0][1][3]


    temp[['Hjorth Mobility','Hjorth Complexity']]      =    temp.Hjorth[0][0],temp.Hjorth[0][1]

    temp.drop(['Hjorth','PSD'],inplace=True,axis=1)


    temp=temp[features_final]  
    
    temps.append(temp)
    
    
    annot=model.predict(temp.values)
    
    
    t = threading.Thread(target=binaural_player,args=(frequencies,annot))
    t.start()
    
    
    
    
    df_prueba.append(df2)
#     print('Data From the Board')
#     display(df.head(10))

#     # demo for data serialization using brainflow API, we recommend to use it instead pandas.to_csv()
#     DataFilter.write_file(data, 'test.csv', 'w')  # use 'a' for append mode
#     restored_data = DataFilter.read_file('test.csv')
#     restored_df = pd.DataFrame(np.transpose(restored_data))
#     print('Data From the File')
#     display(restored_df.head(10))



try:
    while( __name__ == "__main__"):
        main()

except KeyboardInterrupt:
    print("Interrupción manual activada")
    pass

Data From the Board


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
1,1.0,-29346.074682,-76938.548915,-9115.756645,-9095.662426,-20583.632062,-23701.499248,-47370.610818,-2770.699891,0.004,...,32.0,252.0,112.0,31.0,16.0,0.0,0.0,0.0,1.631777e+09,0.0
2,2.0,-20955.743904,-63011.266352,3491.990685,2749.264568,-10192.082547,-12726.211575,-41164.559563,3831.401924,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
3,3.0,-17432.997219,-57562.179275,-47.810381,-294.707751,-12568.967053,-15534.507100,-41604.374838,3343.016308,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
4,4.0,-27300.890064,-73934.541516,-16499.856591,-15760.594995,-26103.529466,-29919.396927,-49553.392774,-5121.902242,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
5,5.0,-30062.828071,-78218.946245,-11265.212150,-11095.763576,-22310.438432,-25543.595915,-48308.981753,-3786.877249,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
6,6.0,-22705.013776,-65805.145002,1631.118552,981.666265,-11770.450982,-14387.192057,-42173.226735,2763.145001,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
7,7.0,-16784.796630,-56481.539486,2629.213349,2166.062852,-10468.126591,-13237.328915,-40553.261704,4454.121525,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
8,8.0,-25585.818361,-71050.697094,-14916.481366,-14226.751235,-24718.011882,-28509.068907,-48583.952914,-4079.975674,0.006,...,48.0,252.0,96.0,31.0,64.0,0.0,0.0,0.0,1.631777e+09,0.0
9,9.0,-30443.522983,-78969.853100,-13238.401799,-12921.364656,-23869.182035,-27227.464584,-49111.252917,-4650.079268,0.006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0


Data From the File


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
1,1.0,-29346.074682,-76938.548915,-9115.756645,-9095.662426,-20583.632062,-23701.499248,-47370.610818,-2770.699891,0.004,...,32.0,252.0,112.0,31.0,16.0,0.0,0.0,0.0,1.631777e+09,0.0
2,2.0,-20955.743904,-63011.266352,3491.990685,2749.264568,-10192.082547,-12726.211575,-41164.559563,3831.401924,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
3,3.0,-17432.997219,-57562.179275,-47.810381,-294.707751,-12568.967053,-15534.507100,-41604.374838,3343.016308,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
4,4.0,-27300.890064,-73934.541516,-16499.856591,-15760.594995,-26103.529466,-29919.396927,-49553.392774,-5121.902242,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
5,5.0,-30062.828071,-78218.946245,-11265.212150,-11095.763576,-22310.438432,-25543.595915,-48308.981753,-3786.877249,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
6,6.0,-22705.013776,-65805.145002,1631.118552,981.666265,-11770.450982,-14387.192057,-42173.226735,2763.145001,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
7,7.0,-16784.796630,-56481.539486,2629.213349,2166.062852,-10468.126591,-13237.328915,-40553.261704,4454.121525,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
8,8.0,-25585.818361,-71050.697094,-14916.481366,-14226.751235,-24718.011882,-28509.068907,-48583.952914,-4079.975674,0.006,...,48.0,252.0,96.0,31.0,64.0,0.0,0.0,0.0,1.631777e+09,0.0
9,9.0,-30443.522983,-78969.853100,-13238.401799,-12921.364656,-23869.182035,-27227.464584,-49111.252917,-4650.079268,0.006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0


Interrupción manual activada


In [6]:
df_prueba[0]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
1,1.0,-29346.074682,-76938.548915,-9115.756645,-9095.662426,-20583.632062,-23701.499248,-47370.610818,-2770.699891,0.004,...,32.0,252.0,112.0,31.0,16.0,0.0,0.0,0.0,1.631777e+09,0.0
2,2.0,-20955.743904,-63011.266352,3491.990685,2749.264568,-10192.082547,-12726.211575,-41164.559563,3831.401924,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
3,3.0,-17432.997219,-57562.179275,-47.810381,-294.707751,-12568.967053,-15534.507100,-41604.374838,3343.016308,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
4,4.0,-27300.890064,-73934.541516,-16499.856591,-15760.594995,-26103.529466,-29919.396927,-49553.392774,-5121.902242,0.004,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7487,64.0,-21863.291784,-64305.387653,-8821.361818,-8399.293828,-19656.504054,-23175.562701,-45474.668798,-738.166361,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
7488,65.0,-30692.141437,-79168.895384,-16643.824177,-15969.539102,-26478.591738,-30115.198209,-50245.201617,-5815.499224,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
7489,66.0,-28312.306501,-74713.768031,-5601.369512,-5768.314692,-17793.709671,-20741.547136,-45826.395849,-1075.431833,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0
7490,67.0,-19046.390837,-59532.507900,5567.059585,4788.928305,-8480.855343,-10951.170141,-40058.505840,5004.957915,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631777e+09,0.0


In [ ]:
df2=df_prueba[0]
df2=df2[[1,2]].copy()
df2.drop([0],axis=0,inplace=True)
df2.reset_index(inplace=True,drop=True)

df2

In [13]:
from scipy import signal

In [29]:
df3=pd.DataFrame(columns=['FpZ'])

df3['FpZ']=(signal.resample(df2[1],3000)+signal.resample(df2[2],3000))/2
df3

,FpZ
0,-42459.099629
1,-47160.534120
2,-45750.263094
3,-46502.519531
4,-46028.929250
...,...
2995,-46942.574517
2996,-46483.111154
2997,-47106.324996
2998,-46156.163384


In [32]:
df3['Epoch']=1
df3

,FpZ,Epoch
0,-42459.099629,1
1,-47160.534120,1
2,-45750.263094,1
3,-46502.519531,1
4,-46028.929250,1
...,...,...
2995,-46942.574517,1
2996,-46483.111154,1
2997,-47106.324996,1
2998,-46156.163384,1


C:\Users\DiegoAlegria\AppData\Roaming\Python\Python38\site-packages\pyeeg-0.4.4-py3.8.egg\pyeeg\detrended_fluctuation_analysis.py:125: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
C:\Users\DiegoAlegria\AppData\Roaming\Python\Python38\site-packages\pyeeg-0.4.4-py3.8.egg\pyeeg\detrended_fluctuation_analysis.py:129: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
C:\Users\DiegoAlegria\AppData\Roaming\Python\Python38\site-packages\pyeeg-0.4.4-py3.8.egg\pyeeg\fractal_dimension.py:22: FutureWarning: `rcond` parameter will change 

FpZ                                   \
            dfa <lambda_0> <lambda_1> <lambda_2>   
Epoch                                              
1      1.139784   1.150847   0.050168   0.050168   

                                                                     \
                                              <lambda_3> <lambda_4>   
Epoch                                                                 
1      ([842025.4084654554, 438368.4755548805, 639441...   0.542001   

                                                   
                                       <lambda_5>  
Epoch                                              
1      (0.0003139504842776334, 3055.556595899091)

,epoch,DFA,HFF,Fisher Info,SVD Entropy,PSD,PFD,Hjorth
0,1,1.139784,1.150847,0.050168,0.050168,"([842025.4084654554, 438368.4755548805, 639441...",0.542001,"(0.0003139504842776334, 3055.556595899091)"


,DFA,HFF,Fisher Info,SVD Entropy,PFD,Power Delta Normalized,Power Theta Normalized,Power Alpha Normalized,Power Beta Normalized,Hjorth Mobility,Hjorth Complexity
0,1.139784,1.150847,0.050168,0.050168,0.542001,0.193291,0.100629,0.146787,0.559293,0.000314,3055.556596


In [34]:
import pyeeg

3055.556595899091

In [60]:
max([0,1,2])

2

In [68]:
temp[features_final].values

array([[1.13978444e+00, 1.15084737e+00, 5.01683075e-02, 5.01683075e-02,
        5.42001030e-01, 1.93290657e-01, 1.00629423e-01, 1.46786705e-01,
        5.59293215e-01, 3.13950484e-04, 3.05555660e+03]])